In [1]:
push!(LOAD_PATH, "../src")
using VariableHeightInvertedPendulum
using PyCall
using PyPlot
using Polynomials
using ForwardDiff
# using ODE

INFO: Recompiling stale cache file /Users/twan/code/julia/VariableHeightInvertedPendulum.jl/packages/lib/v0.4/PyCall.ji for module PyCall.
INFO: Recompiling stale cache file /Users/twan/code/julia/VariableHeightInvertedPendulum.jl/packages/lib/v0.4/Conda.ji for module Conda.
INFO: Recompiling stale cache file /Users/twan/code/julia/VariableHeightInvertedPendulum.jl/packages/lib/v0.4/BinDeps.ji for module BinDeps.
INFO: Recompiling stale cache file /Users/twan/code/julia/VariableHeightInvertedPendulum.jl/packages/lib/v0.4/SHA.ji for module SHA.
INFO: Recompiling stale cache file /Users/twan/code/julia/VariableHeightInvertedPendulum.jl/packages/lib/v0.4/LaTeXStrings.ji for module LaTeXStrings.
INFO: Recompiling stale cache file /Users/twan/code/julia/VariableHeightInvertedPendulum.jl/packages/lib/v0.4/PyPlot.ji for module PyPlot.
INFO: Recompiling stale cache file /Users/twan/code/julia/VariableHeightInvertedPendulum.jl/packages/lib/v0.4/Colors.ji for module Colors.
INFO: Recompiling sta

In [2]:
test = false
g = 9.8
x0 = -0.3
z0 = 1.
zd0 = 0.
zf = 1.
orbital_energies = test ? Dict("Eo_negative" => 0.) : Dict("Eo_negative" => -0.1, "Eo_zero" => 0., "Eo_positive" => 0.1);

In [3]:
function run_scenario(scenario::Scenario; kwargs...)
    filename = "/Users/twan/Documents/MIT/presentations/160626 dynamic walking/video/$(scenario.model.name)_$(scenario.name).mp4"
    sim_movie(scenario.model, scenario.initial_conditions, filename; test = test, kwargs...)
end

run_scenario (generic function with 1 method)

In [4]:
models = []
lipm = LIPM("lipm", g, zf)
push!(models, lipm)
# push!(models, PolynomialVariableHeightModel("slope", g, Poly([zf, 0.2])))
push!(models, PolynomialVariableHeightModel("variable_height", g, Poly([zf, 0.2, 0.5])));

scenarios = Scenario[]
for model in models
    for kv in orbital_energies
        scenario_name = first(kv)
        Eo = last(kv)
        xd0 = velocity_given_orbital_energy(model, x0, Eo)
        initial_conditions = [x0; xd0]
        scenario = Scenario(scenario_name, model, initial_conditions)
        push!(scenarios, scenario)
    end
end

In [5]:
function run_solved_trajectory(name, x0, z0, xd0, zd0; kwargs...)
    traj = zero_orbital_energy_trajectory(x0, z0, xd0, zd0, g, zf)
    trajectory_solved_model = PolynomialVariableHeightModel("trajectory_solved", g, traj)
    run_scenario(Scenario(name, trajectory_solved_model, [x0; xd0]); restrict_ztraj = true, kwargs...)
end

run_solved_trajectory (generic function with 1 method)

In [6]:
for scenario in scenarios
    run_scenario(scenario)
end

LoadError: LoadError: PyError (:PyObject_Call) <type 'exceptions.IOError'>
IOError(32, 'Broken pipe')
  File "/usr/local/lib/python2.7/site-packages/matplotlib/animation.py", line 230, in grab_frame
    dpi=self.dpi, **savefig_kwargs)
  File "/usr/local/lib/python2.7/site-packages/matplotlib/figure.py", line 1565, in savefig
    self.canvas.print_figure(*args, **kwargs)
  File "/usr/local/lib/python2.7/site-packages/matplotlib/backend_bases.py", line 2232, in print_figure
    **kwargs)
  File "/usr/local/lib/python2.7/site-packages/matplotlib/backends/backend_agg.py", line 519, in print_raw
    fileobj.write(renderer._renderer.buffer_rgba())

while loading In[6], in expression starting on line 1

In [7]:
run_solved_trajectory("fast", x0, z0, 1., zd0)
run_solved_trajectory("slow", x0, z0, 0.9, zd0)
run_solved_trajectory("too_slow", x0, z0, 0.8, zd0)

In [8]:
run_solved_trajectory("fast_w_region", x0, z0, 1., zd0; show_region = true)
run_solved_trajectory("too_slow_w_region", x0, z0, 0.8, zd0; show_region = true)

In [7]:
# Variable height at same initial conditions that are too slow for LIPM
Eo = -0.04
xd0 = velocity_given_orbital_energy(lipm, x0, Eo)
kwargs = [(:model_only, true), (:simtime, 1.5), (:show_orbital_energy, false), (:stilltime, 1.), (:fig_size, (4., 3.))]
run_scenario(Scenario("comparison", lipm, [x0; xd0]); kwargs...)
run_solved_trajectory("comparison", x0, z0, xd0, zd0; kwargs...)

LoadError: LoadError: PyError (:PyObject_Call) <type 'exceptions.IOError'>
IOError(32, 'Broken pipe')
  File "/usr/local/lib/python2.7/site-packages/matplotlib/animation.py", line 230, in grab_frame
    dpi=self.dpi, **savefig_kwargs)
  File "/usr/local/lib/python2.7/site-packages/matplotlib/figure.py", line 1565, in savefig
    self.canvas.print_figure(*args, **kwargs)
  File "/usr/local/lib/python2.7/site-packages/matplotlib/backend_bases.py", line 2232, in print_figure
    **kwargs)
  File "/usr/local/lib/python2.7/site-packages/matplotlib/backends/backend_agg.py", line 519, in print_raw
    fileobj.write(renderer._renderer.buffer_rgba())

while loading In[7], in expression starting on line 5

In [12]:
run_solved_trajectory("step_up", -0.5, 0.5, 1.8, 0.)

LoadError: LoadError: PyError (:PyObject_Call) <type 'exceptions.IOError'>
IOError(32, 'Broken pipe')
  File "/usr/local/lib/python2.7/site-packages/matplotlib/animation.py", line 230, in grab_frame
    dpi=self.dpi, **savefig_kwargs)
  File "/usr/local/lib/python2.7/site-packages/matplotlib/figure.py", line 1565, in savefig
    self.canvas.print_figure(*args, **kwargs)
  File "/usr/local/lib/python2.7/site-packages/matplotlib/backend_bases.py", line 2232, in print_figure
    **kwargs)
  File "/usr/local/lib/python2.7/site-packages/matplotlib/backends/backend_agg.py", line 519, in print_raw
    fileobj.write(renderer._renderer.buffer_rgba())

while loading In[12], in expression starting on line 1

In [ ]:
Pkg.update()

INFO: Updating METADATA...
INFO: Updating cache of Compat...
INFO: Updating cache of JSON...
INFO: Computing changes...
INFO: Upgrading Compat: v0.7.18 => v0.7.20
INFO: Upgrading JSON: v0.5.0 => v0.5.1
